In [ ]:
# default_exp exmaples

In [ ]:
#hide
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#hide
!pip install fastcore
from fastcore.all import *
%load_ext autoreload 
%autoreload 2
%cd "/content/drive/MyDrive/Coding/ModelAssistedLabel"

     |████████████████████████████████| 61kB 3.4MB/s 
/content/drive/MyDrive/Coding/ModelAssistedLabel


In [ ]:
#hide
%run '_Synch.ipynb'

# YOLOv5 Model Assisted Labeling

> Work based on https://models.roboflow.com/object-detection/yolov5.

Labeling a new dataset from scratch is tiresome. To label images faster, I train a YOLOv5 model and apply its predictions to new images.

## How to use

### Vanilla image sets

In [ ]:
from ModelAssistedLabel.core import Generation, Defaults

Recursively search a folder (`repo`) that contains images and labels.

In [ ]:
# export
def prepare_split(resource_dir, out_dir = ".", data_yaml=None, MAX_SIZE=5, verbose=True):
  if data_yaml is None:
    data_yaml = Defaults().data_yaml

  g = Generation(repo=resource_dir, out_dir=out_dir, data_yaml=data_yaml)
  g.set_split(MAX_SIZE=MAX_SIZE)

  def tostr(split):
    return [{k: len(v)} for k,v in split.items()]
    
  def sumg(split):
    return sum([list(x.values())[0] for x in tostr(split)])

  if verbose:
    print("summary: ", tostr(g.split))
    print("checksum:", sumg(g.split))

  return g

In [ ]:
# export
def organize(gen, name):
  zipped = g.process_split(name) #create a zip file in the ROOT directory
  local = os.path.basename(zipped)
  os.system(f'unzip "{local}"') #grab data
  #move the contents of the zip file into postion within the ROOT directory
  for splt in ["test/", "train/", "valid/", "data.yaml"]:
    os.system(f"mv '{local[:-4]}/{splt}' .")
    
  #remove zip file
  os.system(f"rm -f -r '{local}'")
  #removed the folder that was taken out of the zip
  os.system(f"rm -f -r '{local[:-4]}'")


In [ ]:
# export 
from ModelAssistedLabel.core import Trainer

def generate_weights(repo, name):
  g = prepare_split(repo)
  organize(gen=g, name=name)
  t = Trainer(name)
  train_path = "yolov5/runs/train"
  ldir = lambda path: set(os.listdir(path))

  before = ldir(train_path)
  t.train(1)
  after = ldir(train_path)

  current = list(after - before)[0]
  return os.path.join(train_path, current)

In [ ]:
# export
from datetime import datetime

def tidy_weights(repo, name, out_dir=None):
  if out_dir is None:
    out_dir = repo
  results_path = generate_weights(repo, name)

  while True:
    now = datetime.now()
    out = "{os.path.basename(results_path)}-{now.strftime('%f')}"
    outfolder = os.path.join(out_dir, moved_filename)
    if not os.path.exists(outfolder):
      break
  os.system(f"mv '{results_path}' '{outfolder}'")
  return outfolder

In [ ]:
repo = "/content/drive/MyDrive/Coding/Roboflow Export (841)"
name = "nospaces"
tidy_weights(repo, name)

summary:  [{'train': 4}, {'valid': 1}, {'test': 0}]
checksum: 5
target/dest /content/drive/MyDrive/Coding/Roboflow Export (841)/images/screenytake-35-jpg-cropped-jpg_jpg.rf.8a59d5d74ecd425f9bfdffba63277cc6.jpg | ./train/images/screenytake-35-jpg-cropped-jpg_jpg.rf.8a59d5d74ecd425f9bfdffba63277cc6.jpg
target/dest /content/drive/MyDrive/Coding/Roboflow Export (841)/labels/screenytake-35-jpg-cropped-jpg_jpg.rf.8a59d5d74ecd425f9bfdffba63277cc6.txt | ./train/labels/screenytake-35-jpg-cropped-jpg_jpg.rf.8a59d5d74ecd425f9bfdffba63277cc6.txt
target/dest /content/drive/MyDrive/Coding/Roboflow Export (841)/images/save_dirrsave_dirrdd31a247c313689d77bbf8fbf2bd7dbac8d44333-jpg-jpg_jpg.rf.ce0b7e55771cab3ea9a210b96e80541f.jpg | ./train/images/save_dirrsave_dirrdd31a247c313689d77bbf8fbf2bd7dbac8d44333-jpg-jpg_jpg.rf.ce0b7e55771cab3ea9a210b96e80541f.jpg
target/dest /content/drive/MyDrive/Coding/Roboflow Export (841)/labels/save_dirrsave_dirrdd31a247c313689d77bbf8fbf2bd7dbac8d44333-jpg-jpg_jpg.rf.ce0b7

'/content/drive/MyDrive/Coding/Roboflow Export (841)/nospaces'

In [ ]:
%run "_Synch.ipynb"

/content/drive/MyDrive/Coding/ModelAssistedLabel
install nbdev: 
build resources: you
Converted 00_ultralytics.ipynb.
Converted 01_split.ipynb.
Converted 02_train.ipynb.
Converted index.ipynb.
converting: /content/drive/My Drive/Coding/ModelAssistedLabel/index.ipynb
converting /content/drive/My Drive/Coding/ModelAssistedLabel/index.ipynb to README.md
git commit -m: successfully wrapped train.py
Executing: git config --local include.path ../.gitconfig
Success: hooks are installed and repo's .gitconfig is now trusted


In [ ]:
ls

00_ultralytics.ipynb  docker-compose.yml  MANIFEST.in          _Synch.ipynb
01_split.ipynb        docs/               ModelAssistedLabel/  test/
02_train.ipynb        index.ipynb         README.md            train/
CONTRIBUTING.md       LICENSE             settings.ini         valid/
data.yaml             Makefile            setup.py             yolov5/
